In [ ]:
# From https://www.kaggle.com/bhavikjain/tps-june-21-eda-models

In [1]:
!pip install tensorflow

In [2]:
!pip install tensorflow-addons

In [87]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import log_loss

import gc
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations,callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import initializers
from sklearn.preprocessing import OrdinalEncoder

from keras.models import Model

In [88]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_features = train_df.drop(['target', 'id'], 1)
test_features = test_df.drop('id', 1)

In [89]:
def do_transform(the_train, the_test):
    full_df = pd.concat([the_train, the_test])
    for column in full_df:
        sorted_df = full_df.sort_values(by=column)
        encoder = OrdinalEncoder()
        encoder.fit(sorted_df[[column]])

        the_train[column + '_oe'] = encoder.transform(the_train[[column]])
        the_test[column + '_oe'] = encoder.transform(the_test[[column]])
        
    return the_train, the_test


train, test = do_transform(train_features, test_features)

In [90]:
print(len(train.columns))
print(len(test.columns))

150
150


In [91]:
targets = pd.get_dummies(train_df['target'])
X_dim = X_dim = np.shape(train)[1]
print(X_dim)

150


In [92]:
def custom_metric(y_true, y_pred):
    y_pred = K.clip(y_pred, 1e-15, 1-1e-15)
    loss = K.mean(cce(y_true, y_pred))
    return loss

cce = tf.keras.losses.CategoricalCrossentropy()

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_custom_metric', min_delta=1e-05, patience=5, verbose=0,
    mode='min', baseline=None, restore_best_weights=True)

plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_custom_metric', factor=0.7, patience=2, verbose=0,
    mode='min')

In [93]:
def conv_model():

    conv_inputs = layers.Input(shape = (X_dim))
    embed = layers.Embedding (input_dim = 354, 
                              output_dim = 7,
                              embeddings_regularizer='l2')(conv_inputs)
    embed = layers.Conv1D(12,1,activation = 'relu')(embed)        
    embed = layers.Flatten()(embed)
    hidden = layers.Dropout(0.3)(embed)
    
    hidden = tfa.layers.WeightNormalization(
                layers.Dense(
                units=32,
                activation ='selu',
                kernel_initializer = "lecun_normal"))(hidden)
    
    output = layers.Dropout(0.3)(layers.Concatenate()([embed, hidden]))
    output = tfa.layers.WeightNormalization(
    layers.Dense(
                units = 32,
                activation='relu',
                kernel_initializer = "lecun_normal"))(output) 
    output = layers.Dropout(0.4)(layers.Concatenate()([embed, hidden, output]))
    output = tfa.layers.WeightNormalization(
    layers.Dense(
                units = 32, 
                activation = 'relu',
                kernel_initializer = "lecun_normal"))(output)
    
    conv_outputs = layers.Dense(
                units = 9, 
                activation ='softmax',
                kernel_initializer ="lecun_normal")(output)
    
    model = Model(conv_inputs,conv_outputs)
    
    return model

In [100]:
import datetime;
import time;

def do_run(run_number):
    print(datetime.datetime.now())
    start = time.time()

    oof_NN_a = np.zeros((train.shape[0],9))
    pred_NN_a_train = np.zeros((train.shape[0],9))
    pred_NN_a_test = np.zeros((test.shape[0],9))

    N_FOLDS = 50
    EPOCH = 60

    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True)

    for fold, (tr_idx, ts_idx) in enumerate(skf.split(train,train.iloc[:,-1])):
        #print(f"\n ====== TRAINING FOLD {fold} =======")

        X_train = train.iloc[:,0:].iloc[tr_idx]
        y_train = targets.iloc[tr_idx]
        X_test = train.iloc[:,0:].iloc[ts_idx]
        y_test = targets.iloc[ts_idx]

        K.clear_session()

        #print("\n-----Convolution model Training----\n")
        model_conv = conv_model()

        model_conv.compile(loss='categorical_crossentropy', 
                                optimizer= 'adam', #keras.optimizers.Adam(learning_rate=2e-4), 
                                metrics=custom_metric)
        
        K.set_value(model_conv.optimizer.learning_rate, 2e-4)
        model_conv.fit(X_train, y_train,
                  batch_size = 256, epochs = EPOCH,
                  validation_data=(X_test, y_test),
                  callbacks=[es, plateau],
                  verbose = 0)

        pred_a = model_conv.predict(X_test) 
        oof_NN_a[ts_idx] += pred_a 
        score_NN_a = log_loss(y_test, pred_a)
        print(f"RUN {run_number} - FOLD {fold} - Score: {score_NN_a}")
        pred_NN_a_test += model_conv.predict(test.iloc[:,0:]) / N_FOLDS 
        pred_NN_a_train += model_conv.predict(train.iloc[:,0:]) / N_FOLDS 

    score_a = log_loss(targets, oof_NN_a)
    print(f"=== FINAL SCORE CONVOLUTION MODEL : {score_a} ===")
    print(datetime.datetime.now())
    minutes = (time.time() - start) / 60
    print('minutes', minutes, '\n')
    
    return pred_NN_a_test, pred_NN_a_train
    

In [101]:
n_runs = 11
test_result, train_result = do_run(1)

for i in range(2, n_runs+1):
    loop_test_result, loop_train_result = do_run(i)
    test_result = test_result + loop_test_result
    train_result = train_result + loop_train_result
    
test_result = test_result / n_runs
train_result = train_result / n_runs

2021-06-28 02:19:15.226071


/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 14 members, which is less than n_splits=50.
  warnings.warn(("The least populated class in y has only %d"


RUN 1 - FOLD 0 - Score: 1.7500080902157351
RUN 1 - FOLD 1 - Score: 1.7384451197898014
RUN 1 - FOLD 2 - Score: 1.7456390560418367
RUN 1 - FOLD 3 - Score: 1.7629970220215618
RUN 1 - FOLD 4 - Score: 1.7092326966375113
RUN 1 - FOLD 5 - Score: 1.739131463920232
RUN 1 - FOLD 6 - Score: 1.7623983172276056
RUN 1 - FOLD 7 - Score: 1.7296956301946194
RUN 1 - FOLD 8 - Score: 1.7455806619315408
RUN 1 - FOLD 9 - Score: 1.7602252573771402
RUN 1 - FOLD 10 - Score: 1.7686445319862105
RUN 1 - FOLD 11 - Score: 1.7310334147484974
RUN 1 - FOLD 12 - Score: 1.7327031864887104
RUN 1 - FOLD 13 - Score: 1.7639899813756346
RUN 1 - FOLD 14 - Score: 1.7377340403641575
RUN 1 - FOLD 15 - Score: 1.737863386251498
RUN 1 - FOLD 16 - Score: 1.7813178463252262
RUN 1 - FOLD 17 - Score: 1.7314485691105947
RUN 1 - FOLD 18 - Score: 1.7369493845056276
RUN 1 - FOLD 19 - Score: 1.7351157227782532
RUN 1 - FOLD 20 - Score: 1.7425142383705825
RUN 1 - FOLD 21 - Score: 1.75106002821913
RUN 1 - FOLD 22 - Score: 1.7709336436185985
RU

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 14 members, which is less than n_splits=50.
  warnings.warn(("The least populated class in y has only %d"


RUN 2 - FOLD 0 - Score: 1.7390485026296227
RUN 2 - FOLD 1 - Score: 1.7427835170594044
RUN 2 - FOLD 2 - Score: 1.7621150085409172
RUN 2 - FOLD 3 - Score: 1.7468529712921008
RUN 2 - FOLD 4 - Score: 1.7550102663999423
RUN 2 - FOLD 5 - Score: 1.763511789832264
RUN 2 - FOLD 6 - Score: 1.727429956048727
RUN 2 - FOLD 7 - Score: 1.7397079683523624
RUN 2 - FOLD 8 - Score: 1.7582825923627243
RUN 2 - FOLD 9 - Score: 1.7296934760119766
RUN 2 - FOLD 10 - Score: 1.7285958187915385
RUN 2 - FOLD 11 - Score: 1.7844862252594904
RUN 2 - FOLD 12 - Score: 1.7501694556968286
RUN 2 - FOLD 13 - Score: 1.7536769050760195
RUN 2 - FOLD 14 - Score: 1.7644232330317609
RUN 2 - FOLD 15 - Score: 1.7351215240219608
RUN 2 - FOLD 16 - Score: 1.7334722299799323
RUN 2 - FOLD 17 - Score: 1.7597656541913749
RUN 2 - FOLD 18 - Score: 1.7688293039649725
RUN 2 - FOLD 19 - Score: 1.7387778335334734
RUN 2 - FOLD 20 - Score: 1.7396433761278167
RUN 2 - FOLD 21 - Score: 1.735690149722621
RUN 2 - FOLD 22 - Score: 1.7591055994639173
R

...

RUN 9 - FOLD 0 - Score: 1.726385811352171
RUN 9 - FOLD 1 - Score: 1.7336500754645094
RUN 9 - FOLD 2 - Score: 1.7360689300866798
RUN 9 - FOLD 3 - Score: 1.7538550220569595
RUN 9 - FOLD 4 - Score: 1.741073085008189
RUN 9 - FOLD 5 - Score: 1.7374278529230505
RUN 9 - FOLD 6 - Score: 1.7459243326876313
RUN 9 - FOLD 7 - Score: 1.7559070586538874
RUN 9 - FOLD 8 - Score: 1.7426679900875315
RUN 9 - FOLD 9 - Score: 1.718645129289478
RUN 9 - FOLD 10 - Score: 1.7568893102877774
RUN 9 - FOLD 11 - Score: 1.7395799117479473
RUN 9 - FOLD 12 - Score: 1.7554993208004162
RUN 9 - FOLD 13 - Score: 1.747813497968018
RUN 9 - FOLD 14 - Score: 1.7398739017911722
RUN 9 - FOLD 15 - Score: 1.7439345654584468
RUN 9 - FOLD 16 - Score: 1.7288912886213512
RUN 9 - FOLD 17 - Score: 1.741748846024275
RUN 9 - FOLD 18 - Score: 1.7619784358618782
RUN 9 - FOLD 19 - Score: 1.7393132331417873
RUN 9 - FOLD 20 - Score: 1.7333183532906695
RUN 9 - FOLD 21 - Score: 1.7437211430165918
RUN 9 - FOLD 22 - Score: 1.7445159406461752
RUN

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 14 members, which is less than n_splits=50.
  warnings.warn(("The least populated class in y has only %d"


RUN 10 - FOLD 0 - Score: 1.7534749600719661
RUN 10 - FOLD 1 - Score: 1.760622378748376
RUN 10 - FOLD 2 - Score: 1.7378667505159975
RUN 10 - FOLD 3 - Score: 1.749911285478156
RUN 10 - FOLD 4 - Score: 1.75036940949969
RUN 10 - FOLD 5 - Score: 1.7612640608730725
RUN 10 - FOLD 6 - Score: 1.7416364498622716
RUN 10 - FOLD 7 - Score: 1.7458448445564136
RUN 10 - FOLD 8 - Score: 1.733784867909737
RUN 10 - FOLD 9 - Score: 1.710636721428251
RUN 10 - FOLD 10 - Score: 1.7384394332906232
RUN 10 - FOLD 11 - Score: 1.742936234618537
RUN 10 - FOLD 12 - Score: 1.7383324570711702
RUN 10 - FOLD 13 - Score: 1.7418850100575947
RUN 10 - FOLD 14 - Score: 1.7484840980516747
RUN 10 - FOLD 15 - Score: 1.7325639225998892
RUN 10 - FOLD 16 - Score: 1.780222717535682
RUN 10 - FOLD 17 - Score: 1.7377553479680792
RUN 10 - FOLD 18 - Score: 1.7223075382271782
RUN 10 - FOLD 19 - Score: 1.7531426100088283
RUN 10 - FOLD 20 - Score: 1.7382479614801705
RUN 10 - FOLD 21 - Score: 1.7391082364078612
RUN 10 - FOLD 22 - Score: 1.

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 14 members, which is less than n_splits=50.
  warnings.warn(("The least populated class in y has only %d"


RUN 11 - FOLD 0 - Score: 1.7209308898532762
RUN 11 - FOLD 1 - Score: 1.7578936775508336
RUN 11 - FOLD 2 - Score: 1.7346417189636267
RUN 11 - FOLD 3 - Score: 1.7481450674338266
RUN 11 - FOLD 4 - Score: 1.7588483508406207
RUN 11 - FOLD 5 - Score: 1.7433668294204399
RUN 11 - FOLD 6 - Score: 1.7497806238827762
RUN 11 - FOLD 7 - Score: 1.7475916482778266
RUN 11 - FOLD 8 - Score: 1.7410806074859575
RUN 11 - FOLD 9 - Score: 1.7674635839993134
RUN 11 - FOLD 10 - Score: 1.7212561290450394
RUN 11 - FOLD 11 - Score: 1.7569845073306933
RUN 11 - FOLD 12 - Score: 1.758069977522362
RUN 11 - FOLD 13 - Score: 1.7530382259585895
RUN 11 - FOLD 14 - Score: 1.755620615543332
RUN 11 - FOLD 15 - Score: 1.757975988222286
RUN 11 - FOLD 16 - Score: 1.7396321902712808
RUN 11 - FOLD 17 - Score: 1.7545328380041756
RUN 11 - FOLD 18 - Score: 1.7366252258121968
RUN 11 - FOLD 19 - Score: 1.760552778155543
RUN 11 - FOLD 20 - Score: 1.7417975265663117
RUN 11 - FOLD 21 - Score: 1.7446104420535267
RUN 11 - FOLD 22 - Score

In [102]:
abs(log_loss(train_df['target'], train_result))

1.7283459834868942

In [103]:
submission = pd.DataFrame(train_result)
submission.columns = ['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8', 'Class_9']
submission['id'] = train_df['id']

submission.to_csv("cnn_ensemble_oe_train.csv", index=False)

In [104]:
submission = pd.read_csv("./sample_submission.csv")
submission['Class_1']=test_result[:,0]
submission['Class_2']=test_result[:,1]
submission['Class_3']=test_result[:,2]
submission['Class_4']=test_result[:,3]
submission['Class_5']=test_result[:,4]
submission['Class_6']=test_result[:,5]
submission['Class_7']=test_result[:,6]
submission['Class_8']=test_result[:,7]
submission['Class_9']=test_result[:,8]

submission.to_csv("submission_cnn_ensemble_oe_test.csv", index=False)